In [ ]:
#PARAMS
BATCH_SIZE = 32
LR = 0.2
DF_SIZE = 1024
EPOCHS = 1

# Importing libraries and dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Concatenate, Dense, Flatten , BatchNormalization,Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import requests
from io import BytesIO
import tensorflow as tf
import cv2

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential

In [ ]:
!gdown 174YvQO7mdg0tXF_e-gLkd0zpcKdwNbSz

Downloading...
From: https://drive.google.com/uc?id=174YvQO7mdg0tXF_e-gLkd0zpcKdwNbSz
To: /content/content_simulation_train.xlsx
100% 56.2M/56.2M [00:00<00:00, 84.1MB/s]


In [ ]:
df = pd.read_excel('content_simulation_train.xlsx')
df = df.sample(DF_SIZE,random_state=42)

# Data Cleaning and pre-processing


In [ ]:
import re
def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'',text)

def get_counts(df):
  mention = "<mention>"
  df['mention_count'] = df['content'].apply(lambda x: x.split().count(mention))
  #
  hyperlink = "<hyperlink>"
  df["hyperlink_count"] = df['content'].apply(lambda x: x.split().count(hyperlink))
  #
  df['hashtag_count'] = df['content'].apply(lambda x: sum([1 for word in x.split() if word.startswith('#')]))
  df["word_count"] = df['content'].apply(lambda x: sum([1 for word in x.split()]))
  df['content'] = df['content'].str.replace('<mention>', '')
  df['content'] = df['content'].str.replace('<hyperlink>', '')
  df['content'] = df['content'].str.replace('<mention>', '')
  df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if not word.startswith('#')]))
  return df

def get_media_urls(df):
  df['image_url'] = df['media'].str.extract(r"fullUrl='(.*?)'")
  import re
  df['video_url'] = df['media'].apply(lambda x: re.findall(r"url='(.*?)'", x)[1] if 'variants' in x and len(re.findall(r"url='(.*?)'", x)) > 1 else None)
  import ast
  df['thumbnail_url'] = df['media'].str.extract(r"thumbnailUrl='(.*?)'")
  df['duration'] = df['media'].apply(lambda x: float(re.search(r"duration=([\d.]+)", x).group(1)) if 'duration' in x else None)
  df['views'] = df['media'].apply(lambda x: int(re.search(r"views=(\d+)", x).group(1)) if re.search(r"views=(\d+)", x) else None)

  return df
def get_link(row):
  if isinstance(row['image_url'],str):
    return row['image_url']
  else:
    return row['thumbnail_url']

def infcorrection(df):
  df['inferred_company'] = df['inferred company']
  df.drop('inferred company',axis = 1)
  return df

import math
def get_views_duration(df):

  views = list(df['views'])
  result_list = [int(value) if not math.isnan(value) else 0 for value in views]
  df['views'] = result_list/df['views'].max()
  duration = list(df['duration'])
  result_list = [int(value) if not math.isnan(value) else 0 for value in duration]
  df['duration'] = result_list/df['duration'].max()

  return df

from datetime import datetime

# Assuming you have a DataFrame named df with a 'datetime' column
# Replace 'your_data.csv' with the actual file path or URL of your data
# df = pd.read_csv('your_data.csv')

# Convert 'datetime' column to datetime format if not already


# Function to create sine and cosine embeddings for time and date
def create_time_embeddings(df, column_name):

    seconds_in_day = 24 * 60 * 60
    seconds_in_year = 365.25 * 24 * 60 * 60

    # Calculate normalized values
    seconds_of_day = (df[column_name].dt.hour * 3600 + df[column_name].dt.minute * 60 + df[column_name].dt.second) / seconds_in_day
    seconds_of_year = (df[column_name] - df[column_name].min()).dt.total_seconds() / seconds_in_year

    # Create embeddings
    df['day_sin'] = np.sin(2 * np.pi * seconds_of_day)
    df['day_cos'] = np.cos(2 * np.pi * seconds_of_day)
    df['year_sin'] = np.sin(2 * np.pi * seconds_of_year)
    df['year_cos'] = np.cos(2 * np.pi * seconds_of_year)

    df['week_sin'] = np.sin((df['day']*2*np.pi)/7)
    df['week_cos'] = np.cos((df['day']*2*np.pi)/7)
    # df['week_sin'] =
    # df['week_cos'] =
    return df

# Apply the function to create embeddings for 'datetime' column
# df = create_time_embeddings(df, 'datetime')

# from datetime import datetime
def uptime(date):
  ref = '2023-11-21 00:00:00'
  refdate = pd.to_datetime(ref)
  reftime = refdate.timestamp()
  return reftime - date.timestamp()
def get_uptime(df):
  df['uptime'] = df['datetime'].apply(uptime)
  return df

def normalize_uptime(df):
  uptime = list(df['uptime'])
  upt = [int(value) if not math.isnan(value) else 0 for value in uptime]
  df['uptime'] = upt/df['uptime'].max()
  return df

def getweekday(date):
  return date.weekday()

def get_weekday_df(df):
  df['datetime'] = pd.to_datetime(df['date'])
  df['day'] = df['datetime'].apply(getweekday)
  return df

In [ ]:
def master_eda(df):
  df = df.dropna(subset=['content'])
  df = get_counts(df)
  df['content'] = df['content'].apply(remove_emojis)
  df = get_media_urls(df)
  df['link'] = df.apply(get_link,axis = 1)
  df = infcorrection(df)
  df = get_views_duration(df)
  df = get_weekday_df(df)
  df = create_time_embeddings(df,'datetime')
  df = get_uptime(df)
  df = normalize_uptime(df)
  return df[['id','likes', 'content', 'username', 'mention_count', 'hyperlink_count', 'hashtag_count',
       'word_count', 'duration',
       'views', 'link', 'inferred_company', 'day_sin', 'day_cos',
       'year_sin', 'year_cos', 'uptime', 'week_sin', 'week_cos']]

In [ ]:
df = master_eda(df)

In [ ]:
df.head()

,id,likes,content,username,mention_count,hyperlink_count,hashtag_count,word_count,duration,views,link,inferred_company,day_sin,day_cos,year_sin,year_cos,uptime,week_sin,week_cos
4941,4942,94,'The preacher loved his big moment – and the c...,Independent,0,2,0,16,0.000000,0.000000,https://pbs.twimg.com/media/DdqTf61V4AAmeXl?fo...,independent,-0.983872,0.178874,0.677669,-0.735367,0.935177,-0.781831,0.623490
51775,51776,23,Walk this way to see the musicians from the 19...,AMCTheatres,0,2,0,20,0.000000,0.000000,https://pbs.twimg.com/media/D2nsg4xVAAA3UQi?fo...,amc,-0.241640,0.970366,0.992678,0.120790,0.790877,0.781831,0.623490
115253,115254,187,Now that looks quite a-peeling! with the all n...,NokiamobileIN,0,1,1,14,0.035117,0.000069,https://pbs.twimg.com/ext_tw_video_thumb/10542...,nokia,0.772872,-0.634562,-0.941487,0.337048,0.863264,0.000000,1.000000
299321,299322,0,Happy birthday to the King—Elvis Presley—born ...,AAASouthJersey,0,1,0,29,0.000000,0.000000,https://pbs.twimg.com/media/DwZ3OjuXQAIUJr4?fo...,aaa,-0.968148,-0.250380,0.120505,0.992713,0.826815,0.781831,0.623490
173570,173571,1,"4 Persons Die, 6 Injured In Niger Auto Crash",IndependentNGR,0,2,0,11,0.000000,0.000000,https://pbs.twimg.com/media/EeRO7pqVAAASHMf?fo...,independent,-0.955085,-0.296333,-0.480541,-0.876972,0.561648,-0.433884,-0.900969


## Standardization of the features

In [ ]:
def calculate_z_scores(features):
    # Convert the list of features to a NumPy array
    features_array = np.array(features)

    # Calculate the mean and standard deviation along the first axis (axis=0)
    mean_values = np.mean(features_array, axis=0)
    std_values = np.std(features_array, axis=0)

    # Calculate the z-scores
    z_scores = (features_array - mean_values) / std_values

    return z_scores


In [ ]:
myfeatures = ['mention_count','hashtag_count','duration','views']
for feature in myfeatures:
    df[feature] = calculate_z_scores(df[feature])

In [ ]:
df.head()

,id,likes,content,username,mention_count,hyperlink_count,hashtag_count,word_count,duration,views,link,inferred_company,day_sin,day_cos,year_sin,year_cos,uptime,week_sin,week_cos
4941,4942,94,'The preacher loved his big moment – and the c...,Independent,-0.469583,2,-0.746951,16,-0.336148,-0.062724,https://pbs.twimg.com/media/DdqTf61V4AAmeXl?fo...,independent,-0.983872,0.178874,0.677669,-0.735367,0.935177,-0.781831,0.623490
51775,51776,23,Walk this way to see the musicians from the 19...,AMCTheatres,-0.469583,2,-0.746951,20,-0.336148,-0.062724,https://pbs.twimg.com/media/D2nsg4xVAAA3UQi?fo...,amc,-0.241640,0.970366,0.992678,0.120790,0.790877,0.781831,0.623490
115253,115254,187,Now that looks quite a-peeling! with the all n...,NokiamobileIN,-0.469583,1,0.088069,14,0.083320,-0.060529,https://pbs.twimg.com/ext_tw_video_thumb/10542...,nokia,0.772872,-0.634562,-0.941487,0.337048,0.863264,0.000000,1.000000
299321,299322,0,Happy birthday to the King—Elvis Presley—born ...,AAASouthJersey,-0.469583,1,-0.746951,29,-0.336148,-0.062724,https://pbs.twimg.com/media/DwZ3OjuXQAIUJr4?fo...,aaa,-0.968148,-0.250380,0.120505,0.992713,0.826815,0.781831,0.623490
173570,173571,1,"4 Persons Die, 6 Injured In Niger Auto Crash",IndependentNGR,-0.469583,2,-0.746951,11,-0.336148,-0.062724,https://pbs.twimg.com/media/EeRO7pqVAAASHMf?fo...,independent,-0.955085,-0.296333,-0.480541,-0.876972,0.561648,-0.433884,-0.900969


# Loading Models

## Load the Resnet 101 Model for feature extraction

In [ ]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input, decode_predictions

# Load the ResNet-101 model pretrained on ImageNet
res_model = ResNet101(weights='imagenet', include_top=True)

for layer in res_model.layers:
  layer.trainable = False

final_img_model = Model(inputs = res_model.input,outputs = res_model.layers[-2].output)



In [ ]:
# final_img_model.summary()

## Load the encoder

In [ ]:
# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = BertModel.from_pretrained("bert-base-uncased")
# def encode(sentences):#list of sentences passed
#   encoded_input = tokenizer(sentences, return_tensors='pt')
#   output = bert_model(**encoded_input)
#   output_tensor = output.last_hidden_state[:,0,:]
#   output_array = output_tensor.detach().numpy()
#   return output_array

from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

def encode(sentences):
    # Add padding and truncation to ensure the same length
    encoded_input = tokenizer(sentences, return_tensors='tf', padding=True, truncation=True)

    # Forward pass through the BERT model
    output = bert_model(**encoded_input)

    # Extract the embeddings from the last hidden layer
    output_tensor = output.last_hidden_state[:, 0, :]

    # Convert the TensorFlow tensor to a NumPy array
    output_array = output_tensor.numpy()

    return output_array

# Example usage
# sentences = ["This is an example sentence.", "Another sentence."]
# encoded_output = encode(sentences)
# print(encoded_output)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

## NIMA model (for image aesthetics score)

In [ ]:
!git clone https://github.com/titu1994/neural-image-assessment

Cloning into 'neural-image-assessment'...
remote: Enumerating objects: 178, done.
remote: Total 178 (delta 0), reused 0 (delta 0), pack-reused 178
Receiving objects: 100% (178/178), 21.51 MiB | 31.78 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [ ]:
%cd neural-image-assessment/

/content/neural-image-assessment


In [ ]:
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf



base_nima_model = MobileNet((None, None, 3), alpha=1, include_top=False, pooling='avg', weights=None)
x = Dropout(0.75)(base_nima_model.output)
x = Dense(10, activation='softmax')(x)

nima_model = Model(base_nima_model.input, x)
nima_model.load_weights('weights/mobilenet_weights.h5')


def get_score(img):
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x)

    scores = nima_model.predict(x, batch_size=1, verbose=0)

    return np.array(scores[0])

In [ ]:
nima_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv1 (Conv2D)              (None, None, None, 32)    864       
                                                                 
 conv1_bn (BatchNormalizati  (None, None, None, 32)    128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, None, None, 32)    0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, None, None, 32)    288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, None, None, 32)    128 

## IIPA model

Note:  this model looks like it uses resnet 50 with no modifications and hence seems redundant, considering we are using resnet 101 for the images already


In [ ]:
%cd /content
!git clone https://github.com/dingkeyan93/Intrinsic-Image-Popularity

/content
Cloning into 'Intrinsic-Image-Popularity'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 196 (delta 6), reused 6 (delta 6), pack-reused 180
Receiving objects: 100% (196/196), 209.13 MiB | 40.89 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [ ]:
%cd /content/Intrinsic-Image-Popularity
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the ResNet50 model
iipa_model = models.resnet50()
iipa_model.fc = torch.nn.Linear(in_features=2048, out_features=1)

# Load the pre-trained weights
checkpoint = torch.load('model/model-resnet50.pth', map_location=device)
iipa_model.load_state_dict(checkpoint)

def getiipa(image_np, iipa_model):

    # Convert the NumPy array to a PIL Image
    image = Image.fromarray(image_np.astype('uint8'), 'RGB')

    # Preprocess the image
    transform = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
    ])
    image = transform(image)

    # Move the input tensor to the same device as the model
    image = image.unsqueeze(0).to(device)
    iipa_model = iipa_model.to(device)

    # Predict IIPA score
    with torch.no_grad():
        iipa_model.eval()
        iipa_score = iipa_model(image).item()

    return iipa_score


/content/Intrinsic-Image-Popularity


# Creating a Regression Generator

In [ ]:
import random, glob, os
import numpy as np
from tqdm.notebook import tqdm
import typing
try :
    from keras.utils import Sequence #   sequence =  keras.utils.Sequence
except:
    from keras.utils.all_utils import Sequence

# ==================================================
class RegressionGenerator(Sequence):
    '''
    Dataset generator for Regression.
    '''
    def __init__(self, df_X : pd.DataFrame, df_Y : pd.DataFrame, width : int = 224, height : int = 224,
                 channels : int = 3, embedding_dim : int = 512, batch_size : int = 32,
                 encoder : Model = encode,
                 features : typing.List = ['content','link','views','mention_count','hashtag_count', 'duration', 'day_sin', 'day_cos', 'week_sin', 'week_cos', 'year_sin', 'year_cos','uptime']) :
        """
        It is expected that the dataframe has already been split. As we are not generating any derivative data,
        there should be not need for different 'types' of generator. And I am lazy.
        """
        self.width, self.height, self.channels = width, height, channels  # Adjust as needed
        self.embedding_dim = embedding_dim  # Adjust as needed
        self.x_df = df_X[features]
        self.y_df = df_Y
        self.features = features
        self.batch_size = batch_size
        self.encoder = encoder
    # ==================================
    def __getitem__(self, index):
        # Disabling 4 threads for now
        batch_x = self.x_df[index*self.batch_size:(index+1)*self.batch_size]
        batch_y = self.y_df[index*self.batch_size:(index+1)*self.batch_size].to_numpy(dtype=float)
        contents = [] #!
        images = []
        others = []

        for index, row in batch_x.iterrows():
          contents.append(row['content'])
          img = self.read_data(row['link'])
          images.append(img)
          nima_score = get_score(img)
          iipa_score = np.array(getiipa(img,iipa_model))
          iipa_score = iipa_score.reshape(1)
          others.append(np.concatenate((iipa_score,nima_score,np.array(row[['mention_count','hashtag_count','duration','views','day_sin','day_cos', 'week_sin', 'week_cos', 'year_sin', 'year_cos', 'uptime']]))))
        images = tf.convert_to_tensor(images)
        others = tf.convert_to_tensor(others)
        contents = tf.convert_to_tensor(self.encoder(contents))

        return [contents, images, others ], batch_y


    def __len__(self):
      return len(self.x_df)//self.batch_size

    def read_data (self, url) :
        '''
        Return the np.array of image @url
        '''
        response = requests.get(url, stream=True)
    # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Convert the image content to a NumPy array
            image_array = np.asarray(bytearray(response.content), dtype=np.uint8)

            # Decode the NumPy array to an OpenCV image
            image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

            # Resize the image to the desired dimensions
            image = cv2.resize(image, (self.width, self.height))

            # Normalize pixel values to the range [0, 1]
            image = image / 255.0

            # Return the preprocessed image
            return image


        return np.zeros((self.width, self.height, self.channels))

# train_gen = RegressionGenerator(df[:19999], df[:19999]['likes'])
# train_gen = RegressionGenerator(df, df['likes'])

##Split

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df, df['likes'], test_size=0.2, random_state=42)

# Display the shape of the resulting sets
print("Training set shape (X, y):", X_train.shape, y_train.shape)
print("Testing set shape (X, y):", X_test.shape, y_test.shape)

Training set shape (X, y): (819, 19) (819,)
Testing set shape (X, y): (205, 19) (205,)


In [ ]:
training_gen = RegressionGenerator(X_train, y_train, batch_size = BATCH_SIZE)
val_gen = RegressionGenerator(X_test, y_test, batch_size = BATCH_SIZE)

# Model Architecture


In [ ]:
def text_processor(embed_dim : int = 768, params = [(128, 'relu')]):
  input_tensor = Input(shape = (embed_dim, ))
  x = input_tensor
  for dim, activ in params:
    x = Dense(dim, activation = activ)(x)
  x = BatchNormalization()(x)
  output_tensor = x
  model = tf.keras.Model(input_tensor, output_tensor)
  return model

In [ ]:
def image_processor(img_dim : np.array = (224,224,3), dense_dim : int = 128):
  input_tensor = final_img_model.input
  x = input_tensor
  x = BatchNormalization()(final_img_model.output)
  output_tensor = Dense(dense_dim, activation = 'relu')(x)
  model = tf.keras.Model(input_tensor, output_tensor)
  return model


In [ ]:
def other_processor(feature_dim : int = 22, params = [(128, 'relu')]):
  input_tensor = Input(shape = (feature_dim, ))
  x = input_tensor
  for dim, activ in params:
    x = Dense(dim, activation = activ)(x)
  x = BatchNormalization()(x)
  output_tensor = x
  model = tf.keras.Model(input_tensor, output_tensor)
  return model

In [ ]:
def regression_model(embed_dim : int = 768, img_dim : typing.Tuple = (224,224,3), params = [(256, 'relu'),(128,'relu'), (1, 'relu')]):
  img_proc = image_processor(img_dim)
  text_proc = text_processor(embed_dim)
  other_proc = other_processor()
  input_tensor = tf.keras.layers.concatenate([text_proc.output, img_proc.output, other_proc.output])

  x = input_tensor
  for dim, activ in params:
    x = Dense(dim, activation = activ)(x)
    x = BatchNormalization()(x)

  x = Dropout(0.2)(x)
  output_tensor = x
  model = tf.keras.Model([text_proc.input, img_proc.input, other_proc.input], output_tensor)

  return model

# Training

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = 'model_checkpoint.keras'  # Path to save the checkpoint
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',  # Monitor validation loss
                             save_best_only=False,  # Save the best model
                             mode='min',  # Mode to determine best model (minimize loss)
                             verbose=1)  # Verbosity level

In [ ]:
reg  = regression_model()

In [ ]:
reg.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss='mean_squared_error', metrics=['mae'])

In [ ]:
%cd /content/drive/MyDrive/Adobe

/content/drive/MyDrive/Adobe


In [ ]:
history = reg.fit(training_gen, validation_data = val_gen, epochs = EPOCHS,callbacks = [checkpoint])

25/25 [==============================] - ETA: 0s - loss: 12393687.0000 - mae: 826.2758 
Epoch 1: saving model to model_checkpoint.keras
25/25 [==============================] - 744s 30s/step - loss: 12393687.0000 - mae: 826.2758 - val_loss: 30070694.0000 - val_mae: 827.2595


In [ ]:
reg.save('modal.keras')

# Evaluation

In [ ]:
[content, image, others], res = val_gen.__getitem__(0)
#bas output check karne ke liye val_gen

In [ ]:
y = reg.predict([content, image, others])

1/1 [==============================] - 8s 8s/step


In [ ]:
np.set_printoptions(suppress=True)
for i in range(len(res)):
  print('prediction_val :' , y[i] , "and true val:" , res[i])

prediction_val : [3.0435698] and true val: 639.0
prediction_val : [3.0435698] and true val: 96.0
prediction_val : [3.0435698] and true val: 25.0
prediction_val : [3.0435698] and true val: 719.0
prediction_val : [3.0435698] and true val: 0.0
prediction_val : [3.0435698] and true val: 8.0
prediction_val : [3.0435698] and true val: 3.0
prediction_val : [3.0435698] and true val: 98.0
prediction_val : [3.0435698] and true val: 1614.0
prediction_val : [3.0435698] and true val: 0.0
prediction_val : [3.0435698] and true val: 1208.0
prediction_val : [3.0435698] and true val: 149.0
prediction_val : [3.0435698] and true val: 112.0
prediction_val : [3.0435698] and true val: 22.0
prediction_val : [3.0435698] and true val: 6.0
prediction_val : [3.0435698] and true val: 143.0
prediction_val : [3.0435698] and true val: 320.0
prediction_val : [3.0435698] and true val: 0.0
prediction_val : [3.0435698] and true val: 94.0
prediction_val : [3.0435698] and true val: 172.0
prediction_val : [3.0435698] and tr